In [1]:
import tensorflow as tf
print(tf.__version__)
import numpy as np 
import pandas as pd 
import joblib
import cufflinks as cf
import plotly.graph_objs as go
import seaborn as sns
import matplotlib.pyplot as plt

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)



2.4.1


In [2]:
with open('/content/drive/MyDrive/data/Enzyme Classification/Train.csv', 'r', encoding='utf-8') as f:
    train = pd.read_csv(f, sep=';|"', engine='python').dropna(how='all', axis=1)
with open('/content/drive/MyDrive/data/Enzyme Classification/Test.csv', 'r', encoding='utf-8') as f:
    test = pd.read_csv(f, sep=',', engine='python').dropna(how='all', axis=1)
with open('/content/drive/MyDrive/data/Enzyme Classification/SampleSubmission.csv', 'r', encoding='utf-8') as f:
    sub = pd.read_csv(f, sep=',', engine='python').dropna(how='all', axis=1)

In [3]:
classmap = {
    "class0" : 0,
    "class1" : 1,
    "class2" : 2,
    "class3" : 3,
    "class4" : 4,
    "class5" : 5,
    "class6" : 6,
    "class7" : 7,
    "class8" : 8,
    "class9" : 9,
    "class10" : 10,
    "class11" : 11,
    "class12" : 12,
    "class13" : 13,
    "class14" : 14,
    "class15" : 15,
    "class16" : 16,
    "class17" : 17,
    "class18" : 18,
    "class19" : 19
}

creaturemap = {
    "creature0" : 0,
    "creature1" : 1,
    "creature2" : 2,
    "creature3" : 3,
    "creature4" : 4,
    "creature5" : 5,
    "creature6" : 6,
    "creature7" : 7,
    "creature8" : 8,
    "creature9" : 9
}

In [4]:
train.loc[:,"LABEL"] = train.LABEL.map(classmap)
train.loc[:,"CREATURE"] = train.CREATURE.map(creaturemap)
train.head(5)

,SEQUENCE_ID,SEQUENCE,CREATURE,LABEL
0,00011UBJ,MPWQELSITVPHEYVEPISYLFGRYGKGVSTELAGNGQVLLRTYLT...,9,11
1,0001HZ3P,MGKLVIGSRGSELALWQANHIKERLKKECLIESEIQIVKTKGDKIL...,3,13
2,0003NJDO,MFIVAVLMLAFLIFVHELGHFTIARICGVKVEVFSIGFGKKLCFFK...,3,16
3,0003XVE3,MKNTSLSTTLTMNDTIAAIATPLGKGAISIIKISGHNALNILKQLT...,3,12
4,00049BS0,MGNRLTKIYTKTGDSGLTGLADGSRTEKDSLRIFVIGDVDELNSLL...,8,13


In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json

In [6]:
BUFFER_SIZE = 10000
BATCH_SIZE = 32
LR = 0.001

In [7]:
vocab_size = 24
embedding_dim = 64
max_length = 500
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
Filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
training_size = int(len(train)*0.8)
print("training_size = ",training_size)
print("testing_size  = ",len(train) - training_size)

training_size =  687021
testing_size  =  171756


In [8]:
training_sentences = train.iloc[0:training_size,1].values
testing_sentences = train.iloc[training_size:,1].values
training_labels = train.iloc[0:training_size,3].values
testing_labels = train.iloc[training_size:,3].values

In [9]:
tokenizer = Tokenizer(num_words=vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', oov_token=oov_tok,lower=False, char_level=True)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [10]:
# Need this block to get it to work with TensorFlow 2.x
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [11]:
# CONVERT TO TF DATASETS

train_dataset = tf.data.Dataset.from_tensor_slices((training_padded,training_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((testing_padded,testing_labels))

train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(train_dataset))
test_dataset = test_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(test_dataset))

In [12]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Save the model with the minimum validation loss
# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint
EarlyStopping_cb = EarlyStopping(monitor='val_loss', mode='min', patience=4,restore_best_weights=True, verbose=1)

# Save best model
Checkpoint_cb = ModelCheckpoint("best_model.h5",save_best_only=True,monitor='val_loss',mode='min')
    
# Reduce learning rate once learning stagnates
# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ReduceLROnPlateau
Reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=2,min_lr=1e-6,mode='min',verbose=1)



AdamOptimizer = tf.keras.optimizers.Adam(learning_rate = LR)
RMSpropOptimizer = tf.keras.optimizers.RMSprop(learning_rate = LR)
SGDOptimizer = tf.keras.optimizers.SGD(learning_rate = LR)
AdagradOptimizer = tf.keras.optimizers.Adagrad(learning_rate = LR)

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, dropout=0.1,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,dropout=0.1)),
    tf.keras.layers.Dense(20, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer=AdamOptimizer,metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 64)           1536      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 500, 128)          66048     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1300      
Total params: 110,100
Trainable params: 110,100
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 25
history = model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset, callbacks=[EarlyStopping_cb, Checkpoint_cb, Reduce_lr])

Epoch 1/25
21470/21470 [==============================] - 1506s 70ms/step - loss: 1.8934 - accuracy: 0.4379 - val_loss: 0.8183 - val_accuracy: 0.7720
Epoch 2/25
21470/21470 [==============================] - 1491s 69ms/step - loss: 0.7599 - accuracy: 0.7901 - val_loss: 0.5056 - val_accuracy: 0.8615
Epoch 3/25
10997/21470 [==============>...............] - ETA: 11:06 - loss: 0.5215 - accuracy: 0.8573

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [ ]:
plot_graphs(history, 'accuracy')

In [ ]:
plot_graphs(history, 'loss')

In [ ]:
sentence = test.SEQUENCE.values
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

padded_sub = np.array(padded)

In [ ]:
preds = model.predict(padded_sub)

In [ ]:
pred_valid_y_labels = np.argmax(preds, axis=-1)
pred_valid_y_labels

In [ ]:
sub["LABEL"] = pred_valid_y_labels
inv_map = {v: k for k, v in classmap.items()}
sub.loc[:,"LABEL"] = sub.LABEL.map(inv_map)

In [ ]:
sub.to_csv("sub1.csv", index=False)

In [ ]:
sub['LABEL'].value_counts(normalize=True).iplot(kind='bar',yTitle='Class', linecolor='black', opacity=0.7,color='red',theme='pearl',
                                                  bargap=0.6,gridcolor='white',title='Distribution of label column in the training set')

In [ ]:
plt.hist(pred_valid_y_labels,bins=20)